# Predicting car prices
I am using [this Kaggle dataset](https://www.kaggle.com/datasets/mystifoe77/car-prices).

<div class="alert alert-block alert-info">
    Links won't work until the link target cells have been executed.
</div>

This is a work-in-progress. I have gotten as far as:
* Preparing the feature set.
* Running a multiple linear regression for heart disease and [reaching a conclusion](#heart_disease_linear_regression_conclusion).
* Running a Naive Bayes analysis and not finding anything useful.
  * [try 1](#bayes_1)
  * [try 2](#bayes_2)
* Running [XGBoost_ed decision trees](#xgboost) and not finding anything useful.

In [35]:
import collections
import csv
import datetime
import itertools
import math
import os
import pickle
import re
import subprocess
from zipfile import ZipFile
# Imports above are standard library
# Imports below are 3rd-party
import numpy as np
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession
import requests
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import recall_score, precision_score, silhouette_score, f1_score, accuracy_score, roc_auc_score, balanced_accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.model_selection import train_test_split, PredefinedSplit, GridSearchCV
from sklearn.naive_bayes import GaussianNB, BernoulliNB, CategoricalNB, ComplementNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.utils.class_weight import compute_sample_weight
from statsmodels.api import qqplot
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.outliers_influence import variance_inflation_factor
import scipy.stats as stats
import statsmodels.api as sm
from xgboost import XGBClassifier, plot_importance

## Download credentials
You will need your own:

In [2]:
# Optional (you can fetch the data manually using the Kaggle link in the first cell):
MY_KAGGLE_USERNAME = "jsf80238"
MY_KAGGLE_KEY = "4036359324650e1c13adfc7ba87ff90e"

### Kaggle API
If you will be downloading from Kaggle you will need your own API key, which you can get by following [these instructions](https://github.com/Kaggle/kaggle-api/blob/main/docs/README.md).

In [19]:
KAGGLE_FILE_NAME = "car_prices.csv"
KAGGLE_DATASET_NAME = "Car_Prices"
KAGGLE_USER_NAME = "mystifoe77"

save_file = "car_prices.df"  # For larger downloads saving the dataframe locally can save time between runs

In [22]:
def get_kaggle_dataset_ref() -> str:
    os.environ["KAGGLE_USERNAME"] = MY_KAGGLE_USERNAME
    os.environ["KAGGLE_KEY"] = MY_KAGGLE_KEY
    import kaggle  # The import itself uses your KAGGLE_KEY, that's why the import is not at the top. Really.
    
    command = "kaggle datasets list --csv --search".split()
    #command.append("NCHS - Death Rates and Causes of Death")
    result = subprocess.run(command + [KAGGLE_DATASET_NAME], capture_output=True)
    data = result.stdout.decode(encoding="utf-8")
    csvreader = csv.DictReader(data.splitlines())
    for row in csvreader:
        title = row["title"]
        user_name, _ = row["ref"].split("/")
        if title == KAGGLE_DATASET_NAME and user_name == KAGGLE_USER_NAME:
            # print(row)
            return row["ref"]
    else:
        raise Exception("Could not find a matching Kaggle dataset.")


In [38]:
spark = SparkSession.builder.appName('car_prices').getOrCreate()
sc = spark.sparkContext

if os.path.exists(save_file):
    pickleRdd = sc.pickleFile(save_file).collect()
    df2 = spark.createDataFrame(pickleRdd)
else:
    dataset_ref = get_kaggle_dataset_ref()
    command = f"kaggle datasets download --unzip --force --file {KAGGLE_FILE_NAME} {dataset_ref}".split()
    result = subprocess.run(command, capture_output=True)
    stdout = result.stdout.decode(encoding="utf-8")
    stderr = result.stderr.decode(encoding="utf-8")
    print(stdout)
    print(stderr)
    print()
    print("Directory contents:")
    print(os.linesep.join(sorted(os.listdir("."))))
    print()
    # Larger downloads are in zip format
    if os.path.exists(KAGGLE_FILE_NAME + ".zip"):
        zipped_object = ZipFile(KAGGLE_FILE_NAME + ".zip")
        zipped_object.extract(KAGGLE_FILE_NAME)
    
    df = spark.read.csv(KAGGLE_FILE_NAME, sep=',', inferSchema=True, header=True)
    # df = pd.read_csv(KAGGLE_FILE_NAME)

    df.rdd.saveAsPickleFile(save_file)

print("Dataframe details:")
print(f"{df.count():,} rows")
print(df.schema)
print(df.limit(10).show())

Dataframe details:
558,837 rows
StructType([StructField('year', IntegerType(), True), StructField('make', StringType(), True), StructField('model', StringType(), True), StructField('trim', StringType(), True), StructField('body', StringType(), True), StructField('transmission', StringType(), True), StructField('vin', StringType(), True), StructField('state', StringType(), True), StructField('condition', IntegerType(), True), StructField('odometer', IntegerType(), True), StructField('color', StringType(), True), StructField('interior', StringType(), True), StructField('seller', StringType(), True), StructField('mmr', IntegerType(), True), StructField('sellingprice', IntegerType(), True), StructField('saledate', StringType(), True)])
+----+---------+-------------------+--------------------+-----------+------------+-----------------+-----+---------+--------+-----+--------+--------------------+-----+------------+--------------------+
|year|     make|              model|                trim